# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import os
os.chdir('..')

from nn.models_tf2 import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies


Loading config file d:\GitHub\ben\src\./config/default.conf
Loading config file d:\GitHub\ben\src\./config/default.conf


## Analyzing a board played on BBO

In [2]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [3]:
display_lin(lin)

In [4]:
board = parse_lin(lin)
print(board)

Board(dealer='S', vuln=[False, True], hands=['9872.A3.KJ973.53', '6QAJ.5628.5.42QJ', 'KT4.J.Q86.AKT976', '53.KQT974.AT42.8'], auction=['1C', '2H', 'X', '4H', '4S', 'PASS', 'PASS', 'PASS'], play=['DA', 'D3', 'D5', 'D6', 'C8', 'C3', 'CJ', 'CA', 'S4', 'S5', 'S8', 'SJ', 'H5', 'HJ', 'HQ', 'HA', 'S2', 'SA', 'ST', 'S3', 'H2', 'SK', 'H4', 'H3', 'D8', 'D2', 'DJ', 'S6', 'SQ', 'C6', 'H7', 'S7', 'H8', 'C7', 'HK', 'S9', 'C5', 'C2', 'CT', 'HT', 'CK', 'H9', 'D7', 'C4', 'DQ', 'D4', 'DK', 'H6', 'D9', 'CQ', 'C9', 'DT'])


In [5]:
card_by_card = CardByCard(*board, models, sampler, False)

In [6]:
await card_by_card.analyze()

analyzing the bidding
1C OK NN-value: 1.200
2H OK NN-value: 1.148
X Suggested bid from NN: CandidateBid(bid=PASS, insta_score=1.1132, expected_score=---, expected_mp=---, expected_imp=---, expected_tricks=---, adjust=---, alert=  , Explain= -- ; 13- HCP))
X NN-values:CandidateBid(bid=X   , insta_score=0.0836, expected_score=---, expected_mp=---, expected_imp=---, expected_tricks=---, adjust=---, alert=  )
4H Suggested bid from NN: CandidateBid(bid=3H  , insta_score=0.6403, expected_score=  178, expected_mp=---, expected_imp= -0.2, expected_tricks= 9.38, adjust=1.54, alert=  , Explain=Preemptive -- 2+!H; 3-8 HCP))
4H NN-values:CandidateBid(bid=4H  , insta_score=0.4117, expected_score=  208, expected_mp=---, expected_imp=  0.1, expected_tricks= 9.47, adjust=0.99, alert=  )
4S Suggested bid from NN: CandidateBid(bid=PASS, insta_score=1.0817, expected_score=  146, expected_mp=---, expected_imp=---, expected_tricks= 7.94, adjust=0.00, alert=  , Explain= -- 3+!C; 6-!D; 5-!H; 5-!S; 11-16 HCP)

Exception: Duplicate cards in deck: AKQJT98765432.AKQJT98765432.AKQJT98765432.AKQJT98765432
   ved BGADLL.PIMC.validateInput() i D:\GitHub\BGA\BGADLL\PIMC.cs:linje 197
   ved BGADLL.PIMC.SetupEvaluation(Hand[] our, Hand oppos, Play current_trick, Play previous_tricks, Constraints[] consts, Player nextToLead, Int32 maxPlayout, Boolean autoplaysingleton, Boolean useStratefy) i D:\GitHub\BGA\BGADLL\PIMC.cs:linje 303

the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [ ]:
card_by_card.cards['S4'].to_dict()

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [ ]:
card_by_card.cards['DA'].to_dict()